## The Meteo module

The meteo module handles the pre-processing of the atmoshpering forcing of the model. 


- Requires:

The data used here can be downladed locally using 

`pytest   `



In [ ]:
#to use the full width of the browser window
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# Not neccessarily needed. Just to check version. Must be > 0.4.1
import pyPoseidon
pyPoseidon.__version__

In [ ]:
import pyPoseidon.meteo as pt

In [ ]:
import datetime
import numpy as np
from glob import glob

In [ ]:
#optional
#import matplotlib.pyplot as plt
#plt.style.use('dark_background')

## Setup

The minimum information required is the geomerty boundary. In the most simple case that is a lat/lon box that defines the area of interest. Without loss of generality we select below Iceland as a test case. Feel free to modify the coordinates. 

In [ ]:
#define in a dictionary the properties of the model..
geometry={'lon_min':-25., # lat/lon window
     'lon_max':-9.,
     'lat_min':56.,
     'lat_max':74.,
    }

### Remote Data

In theory, should be enough. The default setup for the time frame attribute is the current date but this depends on the source of the atmospheric data.

If the source is undefined the process defaults to retrieveing GFS data from an erddap server providing the NCEP Global Best dataset.

In [ ]:
meteo = pt.meteo(**geometry)

In [ ]:
meteo.Dataset

### using GFS

One can provide as url the current gfs url as follows

In [ ]:
start_date = datetime.datetime.today() +  datetime.timedelta(days=-1)
print(start_date.strftime(format='%Y-%m-%d 0:0:0'))

Get the latest available dataset

In [ ]:
r = [0,6,12,18]
h = np.argmin([n for n in [start_date.hour-x for x in r]  if n>0])
r[h]

Constract the url 

In [ ]:
gfs = 'https://nomads.ncep.noaa.gov/dods/gfs_0p25_1hr/gfs{}/gfs_0p25_1hr_{:0>2d}z'.format(start_date.strftime('%Y%m%d'),r[h])
gfs

Now use this in pt ...

In [ ]:
meteo = pt.meteo(url=gfs, **geometry)

In [ ]:
meteo.Dataset

## Local Data

Most of the times atmospheric data will be available locally. More importantly, these data are provided in a forecast mode. 

This means that there is a sequence of files that overlap in terms of time reference as new forecasts become available. 

pyPoseidon provides a way to merge these data in a number of ways.

Below we are using the pytest data (see above).

In [ ]:
DATA_PATH = os.path.dirname(pyPoseidon.__file__)   # get path to the test data installed with the package
DATA_PATH

In [ ]:
# This just a structure to create the list of paths for the actual meteo files - change appropriately 
PATH=DATA_PATH + '/../tests/data/'#Path to meteo files
mpaths = glob(PATH+'uvp*.grib')
mpaths.sort()
mpaths

In [ ]:
dic = {**geometry, 'meteo_source':mpaths, 
            'meteo_engine': 'cfgrib',
            'meteo_merge':'first', 
            'meteo_combine_by' : 'nested',
            'meteo_xr_kwargs':{'concat_dim':'step'} } # The files are forecasting [0-72] so we combine them in terms of time

### retrieve meteo forcing

In [ ]:
#get uvp
meteo = pt.meteo(**dic)

In [ ]:
meteo.Dataset

Additional engine options include 

#### passthrough

In [ ]:
meteo_ = pt.meteo(meteo_source=meteo.Dataset, meteo_engine='passthrough')

In [ ]:
meteo_.Dataset

#### empty

In [ ]:
meteo_= pt.meteo(meteo_engine='empty')

In [ ]:
meteo_.Dataset

You can now attach an already constracted Dataset

In [ ]:
meteo_.Dataset = meteo.Dataset

#### pynio

Not tested recently due to incompatibilities within python dependencies. To be reassessed. 

## Visualise

Since the meteo Dataset is a xarray dataset all integrated viz extentions can be used to visualize the datasets.

#### with matplotlib

In [ ]:
meteo.Dataset.msl[0,:,:].plot() # A 2D graph for one timestep

### Animation

pyPoseidon provides the option to get also animation as

In [ ]:
import pyPoseidon.utils.pplot

In [ ]:
meteo.Dataset.gplot.contourf(x='longitude', y='latitude',z='msl')#, plt_style='dark_background')

### holoviews

In [ ]:
import holoviews as hv
hv.extension('bokeh')
hv.output(widget_location='bottom')

from hvplot import xarray

In [ ]:
#optional for dark theme
#from bokeh.themes import DARK_MINIMAL
#hv.renderer('bokeh').theme = DARK_MINIMAL

In [ ]:
meteo.Dataset.msl.hvplot.contourf(x='longitude',y='latitude',cmap='viridis')

### geoviews

In [ ]:
import geoviews as gv
import geoviews.feature as gf

In [ ]:
g50 = gf.coastline(plot=dict(scale='50m'), style=dict(linewidth=1.5))

In [ ]:
meteo.Dataset.msl.hvplot(x='longitude',y='latitude',geo=True, cmap='viridis', width=800, height=400) * g50 * gf.borders()

## Output to file depending on the solver

In [ ]:
#Create a folder to save the output
import os
if not os.path.exists('test'):
            os.makedirs('test')

In [ ]:
#generic netCDF output
meteo.Dataset.to_netcdf('./test/test.nc') # to netcdf

In [ ]:
#save to DELFT3D format
meteo.to_output(solver='d3d',rpath='./test/') # to u,v,p for d3d

In [ ]:
#save to SCHISM format
meteo.to_output(solver='schism',rpath='./test/')

Additional options include

In [ ]:
# split into more files
meteo.to_output(solver='schism',rpath='./test/', meteo_split_by='day')

In [ ]:
# Define index for meteo
meteo.to_output(solver='schism',rpath='./test/', meteo_split_by='day', m_index=2)